### Loading libraries and files

In [1]:
import pickle
import pandas as pd
import json
import langdetect
import re
import Algorithmia
import bs4 as bs
import lxml
from urllib.request import Request, urlopen
import time
from fuzzywuzzy import process
import tfidf_matcher as tm
from fake_useragent import UserAgent
import numpy as np
import pandas as pd
import recordlinkage

C:\Users\ACER\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# contains correct muckrack urls with either google scraping, tfidf matching or brute force approach.
author_df = pd.read_csv('../Data/journalists_v1.csv')

# original news dataset containing article info 
news_df = pd.read_csv('../Data/articles_v1.csv')

# emails dataset
emails_df = pd.read_csv("../Data/compression_test.txt.gz", compression='gzip',sep = '\t')
emails_df = emails_df.fillna("")
emails_df = emails_df[['email_address', 'first_name_df', 'second_name_df']]

In [3]:
author_df

,author_name_clean,author_article_count,request_url,redirect_url,method,confidence,beats,title,location,links
0,Aja Styles,14,https://muckrack.com/aja-styles,https://muckrack.com/aja-styles,brute force,-,"Arts and Entertainment, Australia",Senior Writer — WAtoday,Perth,https://www.watoday.com.au/by/aja-styles-hve7u...
1,Jake Johnson,33,https://muckrack.com/jake-johnson,https://muckrack.com/jake-johnson,brute force,-,"Politics, U.S.",Staff Writer — Common Dreams,Portland,"https://t.co/yAayy8hArp, http://twitter.com/jo..."
2,Christine Favocci,19,https://muckrack.com/christine-favocci,https://muckrack.com/christine-favocci,brute force,-,-,-,-,-
3,William Rivers Pitt,14,https://muckrack.com/william-rivers-pitt,https://muckrack.com/william-pitt,brute force,-,"Politics, U.S.",Senior Editor and Lead Columnist — Truthout,United States,"http://t.co/CyDafywwFY, http://twitter.com/WRPitt"
4,Amy Goodman,19,https://muckrack.com/amy-goodman,https://muckrack.com/amy-goodman-2,brute force,-,"Fashion, Metro New York",Freelance Writer and Broadcaster — Freelance,Washington,"http://www.amyegoodman.com/, http://www.amyego..."
...,...,...,...,...,...,...,...,...,...,...
3821,Ceimin Burke,15,https://muckrack.com/ceimin-burke,https://muckrack.com/ceimin-burke,brute force,-,-,-,-,-
3822,Harrison Schwartz,10,https://muckrack.com/harrison-schwartz,https://muckrack.com/harrison-schwartz,brute force,-,-,-,-,-
3823,Amanda Hancock,11,https://muckrack.com/amanda-hancock,https://muckrack.com/amanda-hancock,brute force,-,"Arts and Entertainment, U.S. Regional",Reporter — The Gazette (Colorado),Colorado Springs,"https://t.co/h8MDkiNnUB, http://twitter.com/Am..."
3824,Lisa Mcloughlin,10,https://muckrack.com/lisa-mcloughlin,https://muckrack.com/lisa-mcloughlin,brute force,-,"Arts and Entertainment, United Kingdom",Freelance Entertainment and Lifestyle Journali...,London,"https://t.co/2DzUfumx9A, http://twitter.com/it..."


In [4]:
news_df[['site_name']] = news_df[['site_name']].fillna(value = '-')
news_df

,author,date,description,full_text,site_name,title,topic,url,author_article_count,author_name_clean
0,Aja Styles,2020-12-23,Perth mother Clare has found herself mostly co...,Perth mother Clare* has found herself mostly ...,Brisbane Times,'Pack Lego': Perth family caught in hard borde...,entertainment,https://www.brisbanetimes.com.au/national/west...,14,Aja Styles
1,Jake Johnson,2020-12-23,The billâs gifts to the wealthy underscore t...,In late-night votes just hours after nearly 5...,Truthout,Congress Passes COVID Relief With Billions in ...,politics,https://truthout.org/articles/congress-passes-...,33,Jake Johnson
2,Christine Favocci,2020-12-23,It is naive to think that either party is free...,The left has insisted that voter fraud is jus...,The Western Journal,PA Man Facing Charges of Unlawful Voting After...,tech,https://www.westernjournal.com/pa-man-facing-c...,19,Christine Favocci
3,William Rivers Pitt,2020-12-23,What Trump may do in his waning days is only u...,"The endgame being played out by Donald Trump,...",Truthout,What Will Trump Attempt in His Last Days? We M...,politics,https://truthout.org/articles/what-will-trump-...,14,William Rivers Pitt
4,Amy Goodman,2020-12-23,Critics say the $900 billion relief package do...,As Congress passes a $900 billion coronavirus...,Truthout,The Insufficient COVID Stimulus Must Not Be Fo...,business,https://truthout.org/video/the-insufficient-co...,19,Amy Goodman
...,...,...,...,...,...,...,...,...,...,...
119946,Daniel Smith,2020-12-05,Ashley Towne decided to get in shape to marry ...,When you subscribe we will use the informatio...,WalesOnline,'I can't get married looking like this' Rower ...,sport,https://www.walesonline.co.uk/news/uk-news/i-c...,147,Daniel Smith
119947,Victoria Jones,2020-12-05,Experimenting on the ISS allows scientists to ...,When you subscribe we will use the informatio...,WalesOnline,Space experiment could unlock resources for mi...,tech,https://www.walesonline.co.uk/news/uk-news/spa...,92,Victoria Jones
119948,Nisha Mal,2020-12-05,All of the properties are Grade I listed build...,Three buildings featured in a Jane Austen nov...,WalesOnline,Three buildings featured in a Jane Austen nove...,tech,https://www.walesonline.co.uk/news/uk-news/thr...,66,Nisha Mal
119949,Nisha Mal,2020-12-05,"'It's all one big conundrum,' says Sheila Herbert",Woman's home is in Tier 2 while her garden fa...,WalesOnline,Woman's home is in Tier 2 while her garden fal...,tech,https://www.walesonline.co.uk/news/uk-news/wom...,66,Nisha Mal


### Record deduplicating

In [5]:
# make a first name column for blocking using recordlinkage
def first_name(name):
    return name.split(' ')[0].lower()

# get the outlet the author most commonly works for and has the most articles for
def author_outlet(author):
    rows = news_df[news_df.author_name_clean == author]
    return rows.site_name.value_counts().index[0]

# concatenate above 2 fields to compare it with potential email matches
def concatenate_fields(author, outlet):
    return author + ' ' + outlet

In [6]:
def match_disambiguate_deduplicate(authors, emails, key):
    
    author_df['first_name_df'] = author_df.apply(lambda x: first_name(x['author_name_clean']), axis=1)
    author_df['common_outlet'] = author_df.apply(lambda x: author_outlet(x['author_name_clean']), axis=1)
    author_df['author + outlet'] = author_df.apply(lambda x: concatenate_fields(x['author_name_clean'], x['common_outlet']), axis=1)
       
    # create and indexer and block columns by the first name
    indexer = recordlinkage.Index()
    indexer.block('first_name_df')
    candidate_links = indexer.index(authors, emails)
    print("candidate_links:", len(candidate_links))
    
    # compare the author + outlet string from authors dataframe with the email string from the emails dataframe
    compare_cl = recordlinkage.Compare()
    compare_cl.string('author + outlet', 'email_address', label = 'match')
    features = compare_cl.compute(candidate_links, authors, emails)
    
    # find all unique author indexes for which a potential email match is found
    author_indexes = list(set([i[0] for i in features[features.match >= 0.5].index]))

    # to test, take only rows where the match confidence is >= 0.5
    features = features[features.match >= 0.5]
    
    # create an empty dataframe to store matches
    matches = pd.DataFrame (columns = ['author_index', 'email_index', 'match_threshold', 'scraper_response'])

    # for every author, having a variety of matching email indexes, take the one with highest match confidence
    for i in author_indexes:
        temp = features.loc[i].sort_values('match', ascending = False).head(1)
        row = pd.Series(list([i, temp.index[0], round(temp.match.values[0], 3), '-']), index = matches.columns)
        matches = matches.append(row, ignore_index = True) 
    
    # do a google search for every matching email IF they have a valid muckrack url found before, else we cannot confirm
    
    client = Algorithmia.client(key)
    algo = client.algo('specrom/Google_scraper/0.1.4')
    algo.set_options(timeout=300)
    
    for i in range(len(matches)):
        if i % 10 == 0: 
            print(i)
        if authors.iloc[int(matches.iloc[i].author_index)].method == 'no match':
            matches.loc[i, 'scraper_response'] = 'invalid'
            print("no valid muckrack url to compare with", matches.iloc[i].author_index)
        else:
            try:
                email_to_google = emails.iloc[int(matches.iloc[i].email_index)].email_address
                query = {"query": f"{email_to_google} muckrack"}
                results = algo.pipe(query).result
                matches.at[i, 'scraper_response'] = results
            except:
                pass
        
    return matches

In [7]:
email_matches = match_disambiguate_deduplicate(author_df, emails_df, 'simwGAeoL9oH26JJvCS0pBAGzcD1')

candidate_links: 1326051
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550


In [8]:
email_matches

,author_index,email_index,match_threshold,scraper_response
0,6,1764498,0.533,"[{'position': 1, 'snippet': 'Find David Matthe..."
1,8,1141505,0.536,-
2,2066,458863,0.581,"[{'position': 1, 'snippet': 'Find Howard Sinke..."
3,21,617881,0.593,"[{'position': 1, 'snippet': 'Find Brett Lackey..."
4,2076,1064708,0.704,"[{'position': 1, 'snippet': 'Find Vinciane Ngo..."
...,...,...,...,...
549,2008,18613,0.625,[]
550,2009,1586909,0.593,[]
551,2017,6481,0.500,[]
552,2022,23415,0.652,[]


In [9]:
email_matches

,author_index,email_index,match_threshold,scraper_response
0,6,1764498,0.533,"[{'position': 1, 'snippet': 'Find David Matthe..."
1,8,1141505,0.536,-
2,2066,458863,0.581,"[{'position': 1, 'snippet': 'Find Howard Sinke..."
3,21,617881,0.593,"[{'position': 1, 'snippet': 'Find Brett Lackey..."
4,2076,1064708,0.704,"[{'position': 1, 'snippet': 'Find Vinciane Ngo..."
...,...,...,...,...
549,2008,18613,0.625,[]
550,2009,1586909,0.593,[]
551,2017,6481,0.500,[]
552,2022,23415,0.652,[]


### Validating match quality

In [10]:
def found_match(author_index, email_index, scraper_response):
    if not scraper_response or scraper_response == 'invalid' or scraper_response == '-':
        return 'Invalid'
    else:
        # print(author_df.iloc[author_index].redirect_url, scraper_response[0]['url'])
        # print()
        return author_df.iloc[author_index].redirect_url == scraper_response[0]['url']

In [11]:
email_matches['email_checks_out'] = email_matches.apply(lambda x: found_match(x['author_index'], x['email_index'], x['scraper_response']), axis=1)

In [12]:
email_matches[email_matches.email_checks_out == True].match_threshold.mean()

0.6124936708860763

In [13]:
email_matches[email_matches.email_checks_out == False].match_threshold.mean()

0.5405354330708662

In [14]:
email_matches.email_checks_out.value_counts()

Invalid    348
False      127
True        79
Name: email_checks_out, dtype: int64

In [15]:
email_matches[email_matches.email_checks_out == True]

,author_index,email_index,match_threshold,scraper_response,email_checks_out
3,21,617881,0.593,"[{'position': 1, 'snippet': 'Find Brett Lackey...",True
4,2076,1064708,0.704,"[{'position': 1, 'snippet': 'Find Vinciane Ngo...",True
5,2080,365363,0.679,"[{'position': 1, 'snippet': 'Find Caroline Val...",True
8,44,598130,0.581,"[{'position': 1, 'snippet': 'Find Ellie Philli...",True
9,2095,181162,0.545,"[{'position': 1, 'snippet': 'Find Sam Wall of ...",True
...,...,...,...,...,...
384,3281,191863,0.500,"[{'position': 1, 'snippet': 'Currently, Person...",True
386,1238,25105,0.609,"[{'position': 1, 'snippet': 'Find James Boyd o...",True
388,3299,22030,0.517,"[{'position': 1, 'snippet': 'Find Saleen Marti...",True
390,1257,40664,0.630,"[{'position': 1, 'snippet': 'Find Mark Sweney ...",True


In [16]:
# example
author_df.iloc[21].author_name_clean, author_df.iloc[21].common_outlet, emails_df.iloc[617881]

('Brett Lackey',
 'Mail Online',
 email_address     brett.lackey@mailonline.com
 first_name_df                           brett
 second_name_df                         lackey
 Name: 617881, dtype: object)

In [17]:
email_matches[email_matches.email_checks_out == 'Invalid'].match_threshold.mean()

0.5717097701149417

In [18]:
email_matches.reset_index(drop = True).to_csv('../Data/email_matches.csv', index = False)

In [21]:
author_df['email'] = np.nan
author_df

In [33]:
for i in range(len(email_matches)):
    cell = email_matches.iloc[i]
#     print(cell.email_checks_out)
    if cell.email_checks_out == 'True':
#         print(emails_df.loc[cell.email_index, 'email_address'])
        author_df.loc[cell.author_index, 'email'] = emails_df.loc[cell.email_index, 'email_address']

In [34]:
author_df.email.value_counts()

eloise.valadez@nwi.com               1
michelle_hunter@nola.com             1
alison.medley@chron.com              1
rhett.morgan@tulsaworld.com          1
beth.heyn@heavy.com.                 1
                                    ..
laurence.hammack@roanoke.com         1
levi.parsons@mailonline.com          1
myra.butterworth@mailonline.co.uk    1
ryan.kneller@mcall.com               1
ginnie.graham@tulsaworld.com         1
Name: email, Length: 79, dtype: int64

In [35]:
author_df

,author_name_clean,author_article_count,request_url,redirect_url,method,confidence,beats,title,location,links,email
0,Aja Styles,14,https://muckrack.com/aja-styles,https://muckrack.com/aja-styles,brute force,-,"Arts and Entertainment, Australia",Senior Writer — WAtoday,Perth,https://www.watoday.com.au/by/aja-styles-hve7u...,NaN
1,Jake Johnson,33,https://muckrack.com/jake-johnson,https://muckrack.com/jake-johnson,brute force,-,"Politics, U.S.",Staff Writer — Common Dreams,Portland,"https://t.co/yAayy8hArp, http://twitter.com/jo...",NaN
2,Christine Favocci,19,https://muckrack.com/christine-favocci,https://muckrack.com/christine-favocci,brute force,-,-,-,-,-,NaN
3,William Rivers Pitt,14,https://muckrack.com/william-rivers-pitt,https://muckrack.com/william-pitt,brute force,-,"Politics, U.S.",Senior Editor and Lead Columnist — Truthout,United States,"http://t.co/CyDafywwFY, http://twitter.com/WRPitt",NaN
4,Amy Goodman,19,https://muckrack.com/amy-goodman,https://muckrack.com/amy-goodman-2,brute force,-,"Fashion, Metro New York",Freelance Writer and Broadcaster — Freelance,Washington,"http://www.amyegoodman.com/, http://www.amyego...",NaN
...,...,...,...,...,...,...,...,...,...,...,...
3821,Ceimin Burke,15,https://muckrack.com/ceimin-burke,https://muckrack.com/ceimin-burke,brute force,-,-,-,-,-,NaN
3822,Harrison Schwartz,10,https://muckrack.com/harrison-schwartz,https://muckrack.com/harrison-schwartz,brute force,-,-,-,-,-,NaN
3823,Amanda Hancock,11,https://muckrack.com/amanda-hancock,https://muckrack.com/amanda-hancock,brute force,-,"Arts and Entertainment, U.S. Regional",Reporter — The Gazette (Colorado),Colorado Springs,"https://t.co/h8MDkiNnUB, http://twitter.com/Am...",NaN
3824,Lisa Mcloughlin,10,https://muckrack.com/lisa-mcloughlin,https://muckrack.com/lisa-mcloughlin,brute force,-,"Arts and Entertainment, United Kingdom",Freelance Entertainment and Lifestyle Journali...,London,"https://t.co/2DzUfumx9A, http://twitter.com/it...",NaN


In [36]:
author_df.to_csv('../Data/journalists_v2.csv', index = False)